In [1]:
!pip install captum
!pip install transformers
!pip install shap
!pip install zhon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 4.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 81.1 MB/s 
     |████████████████████████████████| 6.6 MB 70.2 MB/s 
     |████████████████████████████████| 101 kB 12.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 569 kB 5.2 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 99 kB 4.5 MB/s 
  Created wheel for zhon: filename=zhon-1.1.5-py3-none

In [2]:
from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from transformers import AutoModelForSequenceClassification, AdamW, BertConfig, AutoTokenizer
from zhon.hanzi import punctuation

In [3]:
######test##############
import numpy as np
import pandas as pd
import time
import re

In [4]:
import transformers
import shap
# import jieba
import re
# from zhon.hanzi import punctuation
import torch

## 导入数据

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
txt=pd.read_csv("/content/drive/MyDrive/情感打分数据包/日文/日文情感标签总.csv")
#txt = '東京金融市場が年明けから波乱の展開となったのは、「中国経済の悪化」「中東情勢の緊迫」「米国株の変調」という三つの不安材料が表れたためだ。昨年までは金融緩和と国内企業の好業績を背景に、円安・株高の好循環が続いてきた。今年の市場は、海外情勢の不透明感がこれまで以上に影響するとみられ、円高・株安基調に転じるとの見方も少なくない。'
# txt = '湜辛各地报纸向农民宣传总路线的重要经验之一，就是必须善于从农民的切身利益和实际经验出发，来教育农民认识党的号召和政策的正确性。我们的报纸要在广大农民群众走社会主义道路的积极性空前高涨的基础上，进一步引导农民发展互助合作组织，同样应该运用这个经验。由于农业生产合作社“方向好、产量高、收入多”，它在目前整个互助合作运动中日益显出重要的地位，并日益变成为党领导互助合作运动继续前进的重要环节，加强对农业生产合作社的宣传，就具有特别重大的意义。在这一方面，山西日报近两个多月来对农业互助合作特别是对农业生产合作社优越性的宣传，是一个良好的开端。要吸引农民积极参加互助合作，只能采取说服教育和典型示范的办法。“示范是在多方面的，但一切事情需要能够真正做到提高生产率，达到多产粮食或其他作物、增加收入这一个目的。只有在多产粮食增加收入这样的号召下，才可能动员农民组织起来。”（一九五三年二月公布的“中国共产党中央委员会关于农业生产互助合作的决议”）山西日报依据这一指示的精神，结合山西的实际情况，以巩固与发展农业生产合作社作为开展冬季农业生产的中心工作，从秋收分配的结果入手，突出地宣传了农业生产合作社的优越性。报纸一方面综合报道了一九五三年全省以及一个专区、一个县的农业生产合作社收获量较之单干农民和互助组显著增加的巨大成就，另一方面广泛介绍了不同地区、不同条件的农业生产合作社提高生产、增加收入的典型事例。例如，十一月三日报道的“武侯梨社今年‘四喜’齐来”，就说明平顺县武侯梨农业生产合作社由于发挥了集体经营的长处，全面地组织社员生产，不仅在农业生产方面，而且在林业、畜牧、副业生产方面都达到全面增产，因而社员收入大为增加。十一月二十一日报道的“赵城陈光亮农业社粮食打得多”，说明该社由于合理地使用了土地、劳力，提高了耕作技术，一九五三年每亩秋田平均打粮两石零四升八合，超过村里最好的陈光生互助组每亩平均产量一斗三升八合，比最好的单干农民陈九儿每亩多收三斗一升八合。十一月二十五日报道的“从三百六十三斤到五百三十三斤”，说明壶关县冯海科农业生产合作社虽处于大部是坡地和梯田的山区条件下，再加上连年有旱灾和洪灾的威胁，仍然连续三年增产，一九五三年的产量超过了抗日战争前生产水平的百分之二百九十五，比一九五○年还是互助组的时候提高了百分之九十七点四。报纸通过这些典型事例，不仅确切地证明了农业生产合作社能够比互助组和单干农民多打粮食，而且具体地揭示了它所以能多打粮食的原因，即由于农业生产合作社内部社会主义因素的增长，由于统一经营土地和统一使用劳力，从而便于解决生产资料的困难，便于改进和提高耕作技术，便于战胜各种自然灾害，这就能在相当的程度上克服小农经济的弱点，并能解决互助组中难以解决的一些矛盾。山西日报在大力宣传农业生产合作社增产粮食、增加社员收入的典型事例的同时，还有效地采用了农民“现身说法”的方式，以农民参加互助合作运动特别是参加农业生产合作社的亲身体验，来向农民进行自我教育。十一月八日发表的“‘四愁’变成四不怕”，就是通过浑源县许武乡农业生产合作社社员翟树权的自述，对单干和合作作了这样的比较：过去单干时由于劳力少因而有“四愁”——愁种、愁浇、愁锄、愁收；加入农业生产合作社后，“人多力量大，集体经营，处处做得周到”，种、浇、锄、收，再也不用愁了。十二月五日发表的武乡县禄村乡农业生产合作社社员张来元的来信，又说明老年农民张来元在土地改革中虽分到十八亩土地，但因年纪大，人手少，“年年不能按时下种，秋后总不能多打粮食”；一九五一年参加了互助组，解决了单干时的不少困难，“不但多打了粮食，而且也不像单干时那样忙忙乱乱地连口气也喘不过来了”；一九五二年冬天，他入了农业生产合作社后，不但能够从事适合自己体力的劳动，而且一年下来“比互助组多收入十三石（粮食），比单干时多收入十七石，全家生活大大地改善了”，因此张来元“决心永远听毛主席的话，跟着共产党走”。报纸发表的各地加入农业生产合作社的不同农户的亲身体验，既从多方面进一步阐明了农业生产合作社的优越性，又为广大农民树立了走互助合作道路达到共同富裕的生动榜样，这对于农民群众显然是一种实际而有效的教育。山西日报在宣传农业生产合作社的优越性时，还通过关于互助合作的专栏，运用事实对比和具体算账的方法，针对农民中对农业生产合作社的误解和怀疑，进行系统的分析批判。从十一月下旬以来，该报关于互助合作的专栏就曾连续而集中地批判了部分农民怕参加农业生产合作社“不自由”，劳力强、技术好的农民认为不搞互助合作一样能够提高生产，地多劳力少的农民顾虑入社后会吃亏，生产条件比较好、生活也比较富裕的农民企图单干发财等各种想法。报纸在进行这种批判时，都曾选择典型的人和典型的事来加以对比，并配合以“三言两语”的小言论。例如，十二月八日就曾以兴县石岭村白各嗟和猗氏县上朝村陈喜顺这两个劳力强、技术好的人，前者参加农业生产合作社而多分了粮食，后者退出农业生产合作社而少收了棉花的事实，来“给劳力强技术好的人算账”。事实证明，单干的困难很多，而农业生产合作社却能够克服单干所不能克服的许多困难，因而劳力强、技术高的人也只有参加农业生产合作社才能走向富裕；而且由于农业生产合作社以劳力分红为主，越是劳力强技术好的人越该入社来发挥自己的特长。十二月二十三日又以闻喜县五区东杜村的柴银锁和柴德洪这两个土地多劳力少的人，前者参加了农业生产合作社结果比往年多收入一千多斤粮食，后者入了社又退出来，结果比同样条件的社员农户少收二百多斤粮食的事实，来进行对比。报纸在分析了农业生产合作社统一经营土地因而能够做到“地尽其利”后，指出地多劳力少的农户参加农业生产合作社也是有利的。报纸通过具有说服力量的事实，来系统地解决农民中对农业生产合作社的各种误解和顾虑，也就更加深入而具体地显示了农业生产合作社优越性的各个方面。农业生产合作社不仅是一种比较先进的农业生产组织，也是党对农民进行现实的社会主义教育的学校。农业生产合作社的社员由于集体劳动和统一使用生产资料，有着较高的政治觉悟，他们不仅在生产上优于单干农民，而且在政治上也是农村中热烈响应党和政府各项号召、积极贯彻党和政府各项政策的基本力量。山西日报在宣传农业生产合作社的优越性时，没有忽视这一个重要的方面。报纸在购粮报道中，曾着重地宣扬了各地农业生产合作社社员认识到国家建设利益和个人利益的一致性从而带头出卖余粮的模范行动。十二月二十七日报道的大同县陈家庄农业生产合作社带领困难农民一同走向富裕，更具体地叙述了该社社员在认识提高的基础上，发扬互助友爱精神，吸收全村二十一户困难农民入社，帮助他们克服困难，引导大家“一同好好往社会主义走”的情况。这就正确地贯彻了使全体农民经过互助合作达到共同富裕的政策。由于山西日报具体而突出地宣传了农业生产合作社的优越性，就使得这一宣传收到较好的效果。一般读者认为这一宣传说服力比较强，教育意义比较大，适合农民的兴趣。应县七区胡峪口村党支部书记王建勋就写信给报社说，他看了报纸知道很多农业生产合作社都比互助组强，又听了总路线总任务的报告，决心和全村互助组组员商量，建立一个农业生产合作社。目前各地正在冬季农业生产工作中积极地贯彻党中央关于发展农业生产合作社的决议。山西日报采用通俗易懂而为农民所容易接受的办法教育农民认识参加农业生产合作社的好处，使农民乐于参加农业生产合作社。这种办法是值得重视的。自然，这不是说我们的报纸可以忽视对互助合作的初级形式—互助组的宣传，或忽视对单干农民的政治教育和生产教育。应该了解，宣传农业生产合作社优越性的目的，正是为了以具体的实际的榜样来吸引广大农民倾向社会主义；正是为了巩固和发展农业生产合作社，并带动互助合作的大发展；正是为了鼓舞千百万群众积极走上互助合作的光明大道，来共同为实现国家对农业的社会主义改造而奋斗'
#txt1 = "宇宙環境の維持などをうたう探査の指針を、国際宇宙探査フォーラム（ＩＳＥＦ）でまとめる背景には、中国やインドなどの新興国や、民間企業の探査が活発化している現状がある。\u3000冷戦時代の米ソに象徴される、国家間の競争の場だった宇宙開発は現在、産業振興など実利が見込める段階にある。ただ、参加者の増加につれて利害がぶつかる恐れが高まり、新たなルールを作る必要が出てきた。\u3000指針は文部科学省が中心となり、各国と調整を進めて作成した。拘束力はないが、政府関係者は「守らねば国際社会の信用を失う。今後の議論の出発点となる『東京原則』だ」と話す。\u3000月の資源探査などが今後具体化した場合、詳細な国際ルールや法整備が不可欠だ。宇宙産業の健全な発展に直結するだけに、日本政府には今後も積極的な関与が求められる。"
#df_sample = pd.DataFrame(columns=['content'])
#df_sample["content"] = txt.txt_seg
#df_sample.loc[1] = [txt1]
df_sample=txt.iloc[12000:14000,:]
df_sample.index=range(len(df_sample))
df_sample

,txt,year,text_length,txt_seg,Topic,Doc_ID,Score_Negative,Score_Neutral,Score_Positive
0,中国遠征へ向けて合宿中のサッカー２１歳以下（Ｕ―２１）日本代表は３日、反町新監督のもとで初め...,2006.08.04,300,中国 遠征 向け 合宿 中 サッカー 歳 以下 日本 代表 日 反町 新 監督 もと 初めて...,24,48297,0.372833,0.429042,0.198125
1,◇単位＝円◆米ドル（銀行間直物）午後５時現在９５．２３〜９５．２６前営業日比△１．５０高値９...,2008.11.21,317,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,18,48298,0.224488,0.666766,0.108745
2,ペプシカップ２００２年世界少年野球選手権最終日（１２日・大阪ドームほか＝読売新聞大阪本社など...,2002.08.13,369,ペプシカップ 年 世界 少年 野球 選手権 最終 日 日 大阪ドーム ほか 読売新聞 大阪 ...,24,48301,0.204547,0.493237,0.302216
3,２０２０年の東京五輪に向け、ポルトガル代表の陸上チームが大分市で事前キャンプをすることが決ま...,2018.10.17,407,年 東京 五輪 向け ポルトガル 代表 陸上 チーム 大分 市 事前 キャンプ する こと ...,24,48303,0.271182,0.509268,0.219550
4,【ワシントン＝中島健太郎】米国の情報機関を統括するクラッパー国家情報長官は１１日、下院情報特...,2013.04.12,553,ワシントン 中島 健太郎 米国 情報 機関 統括 する クラッパー 国家 情報 長官 日 下...,87,48308,0.438101,0.408954,0.152945
...,...,...,...,...,...,...,...,...,...
1995,大蔵省は四日、九三年の地域別国際収支を発表した。それによると、対東南アジア（中国など社会主義...,1994.07.05,283,大蔵省 四 日 九 三 年 地域 別 国際 収支 発表 し た それ よる 対 東南アジア ...,99,56368,0.395083,0.452387,0.152530
1996,第１１管区海上保安本部（那覇市）によると、２６日午後７時現在、中国海警局の公船４隻が、沖縄県...,2017.09.27,85,第 管区 海上 保安 本部 那覇 市 よる 日 午後 時 現在 中国 海 警局 公 船 隻 ...,12,56381,0.286377,0.570069,0.143554
1997,◇単位＝円◆米ドル（銀行間直物）午後５時現在１０９．３６〜１０９．３７前営業日比▼０．５８高...,2019.01.19,420,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,8,56382,0.233884,0.639498,0.126618
1998,山口市の維新百年記念公園陸上競技場で１０日に開催される第１回田島直人記念陸上競技大会（読売新...,2004.10.02,281,山口 市 維新 百 年 記念 公園 陸上 競技 場 日 開催 さ れる 第 回 田島 直人 ...,24,56388,0.355018,0.475439,0.169543


In [34]:
#import string
#result = string.punctuation

In [27]:
df_sample['nopunc'] = df_sample.txt.apply(lambda x:re.sub("[{}]+".format(punctuation), "", x))
df_sample


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,txt,year,text_length,txt_seg,Topic,Doc_ID,Score_Negative,Score_Neutral,Score_Positive,nopunc
0,中国遠征へ向けて合宿中のサッカー２１歳以下（Ｕ―２１）日本代表は３日、反町新監督のもとで初め...,2006.08.04,300,中国 遠征 向け 合宿 中 サッカー 歳 以下 日本 代表 日 反町 新 監督 もと 初めて...,24,48297,0.372833,0.429042,0.198125,中国遠征へ向けて合宿中のサッカー２１歳以下Ｕ―２１日本代表は３日反町新監督のもとで初めての練...
1,◇単位＝円◆米ドル（銀行間直物）午後５時現在９５．２３〜９５．２６前営業日比△１．５０高値９...,2008.11.21,317,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,18,48298,0.224488,0.666766,0.108745,◇単位円◆米ドル銀行間直物午後５時現在９５．２３９５．２６前営業日比△１．５０高値９４．９７...
2,ペプシカップ２００２年世界少年野球選手権最終日（１２日・大阪ドームほか＝読売新聞大阪本社など...,2002.08.13,369,ペプシカップ 年 世界 少年 野球 選手権 最終 日 日 大阪ドーム ほか 読売新聞 大阪 ...,24,48301,0.204547,0.493237,0.302216,ペプシカップ２００２年世界少年野球選手権最終日１２日・大阪ドームほか読売新聞大阪本社など主催...
3,２０２０年の東京五輪に向け、ポルトガル代表の陸上チームが大分市で事前キャンプをすることが決ま...,2018.10.17,407,年 東京 五輪 向け ポルトガル 代表 陸上 チーム 大分 市 事前 キャンプ する こと ...,24,48303,0.271182,0.509268,0.219550,２０２０年の東京五輪に向けポルトガル代表の陸上チームが大分市で事前キャンプをすることが決まり...
4,【ワシントン＝中島健太郎】米国の情報機関を統括するクラッパー国家情報長官は１１日、下院情報特...,2013.04.12,553,ワシントン 中島 健太郎 米国 情報 機関 統括 する クラッパー 国家 情報 長官 日 下...,87,48308,0.438101,0.408954,0.152945,ワシントン中島健太郎米国の情報機関を統括するクラッパー国家情報長官は１１日下院情報特別委員会...
...,...,...,...,...,...,...,...,...,...,...
1995,大蔵省は四日、九三年の地域別国際収支を発表した。それによると、対東南アジア（中国など社会主義...,1994.07.05,283,大蔵省 四 日 九 三 年 地域 別 国際 収支 発表 し た それ よる 対 東南アジア ...,99,56368,0.395083,0.452387,0.152530,大蔵省は四日九三年の地域別国際収支を発表したそれによると対東南アジア中国など社会主義国家を除...
1996,第１１管区海上保安本部（那覇市）によると、２６日午後７時現在、中国海警局の公船４隻が、沖縄県...,2017.09.27,85,第 管区 海上 保安 本部 那覇 市 よる 日 午後 時 現在 中国 海 警局 公 船 隻 ...,12,56381,0.286377,0.570069,0.143554,第１１管区海上保安本部那覇市によると２６日午後７時現在中国海警局の公船４隻が沖縄県石垣市の尖...
1997,◇単位＝円◆米ドル（銀行間直物）午後５時現在１０９．３６〜１０９．３７前営業日比▼０．５８高...,2019.01.19,420,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,8,56382,0.233884,0.639498,0.126618,◇単位円◆米ドル銀行間直物午後５時現在１０９．３６１０９．３７前営業日比▼０．５８高値１０９...
1998,山口市の維新百年記念公園陸上競技場で１０日に開催される第１回田島直人記念陸上競技大会（読売新...,2004.10.02,281,山口 市 維新 百 年 記念 公園 陸上 競技 場 日 開催 さ れる 第 回 田島 直人 ...,24,56388,0.355018,0.475439,0.169543,山口市の維新百年記念公園陸上競技場で１０日に開催される第１回田島直人記念陸上競技大会読売新聞...


## 定义模型

In [28]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [29]:
#MODEL = "../model/twitter-xlm-roberta-base-sentiment" # use this to finetune the language model
MODEL = "cardiffnlp/twitter-xlm-roberta-base-sentiment" # use this to finetune the sentiment classifier
MAX_TRAINING_EXAMPLES = -1 # set this to -1 if you want to use the whole training set

In [30]:
!pip install --no-cache-dir transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [31]:
# tokenizer = BertTokenizer.from_pretrained('model/bert-base-chinese-FineTuned-Binary-Best')
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)

In [32]:
num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels)
model.to(device)
model.eval()
model.zero_grad()

In [33]:
def predict(inputs, attention_mask=None):
    output = model(inputs,attention_mask=attention_mask, )
    return output.logits

def squad_pos_forward_func(inputs, attention_mask=None):
    pred = predict(inputs,
                   attention_mask=attention_mask)
#     pred = pred[position]
    return pred.max(1).values

In [34]:
ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [35]:
def construct_input_ref_pair1(length, ref_token_id, sep_token_id, cls_token_id):
    ref_input_ids = [cls_token_id] + [ref_token_id] * length + [sep_token_id]
    return torch.tensor([ref_input_ids], device=device)

In [36]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    return attributions

## 开始评分

In [ ]:
lig = LayerIntegratedGradients(squad_pos_forward_func, model.roberta.embeddings)
final_result1 = pd.DataFrame()
start = time.perf_counter()
for j in range(len(df_sample.nopunc)):
    e2 = tokenizer(df_sample.nopunc[j], return_tensors='pt')
    print("now is "+str(j)+" -th news")
    input_ids = e2.input_ids
    attention_mask = e2.attention_mask
    ref_input_ids = construct_input_ref_pair1(input_ids.shape[-1]-2, ref_token_id, sep_token_id, cls_token_id)
    
    indices = input_ids[0].detach().tolist()
    all_tokens = tokenizer.convert_ids_to_tokens(indices)
    
    attributions, delta = lig.attribute(inputs=input_ids,
                                  baselines=ref_input_ids,
                                  additional_forward_args=(attention_mask),
                                  return_convergence_delta=True)

    attributions_sum = summarize_attributions(attributions)
    
    ll = pd.DataFrame(all_tokens,columns=['Char'])
    ll['Score'] = attributions_sum.numpy()
    
    final_result1 = pd.concat([final_result1, ll])


end = time.perf_counter()
print('Running time: %s Seconds' % (end - start))

final_result1

now is 0 -th news
now is 1 -th news
now is 2 -th news
now is 3 -th news
now is 4 -th news
now is 5 -th news
now is 6 -th news
now is 7 -th news
now is 8 -th news
now is 9 -th news
now is 10 -th news
now is 11 -th news
now is 12 -th news
now is 13 -th news
now is 14 -th news
now is 15 -th news
now is 16 -th news
now is 17 -th news
now is 18 -th news
now is 19 -th news
now is 20 -th news
now is 21 -th news
now is 22 -th news
now is 23 -th news
now is 24 -th news
now is 25 -th news
now is 26 -th news
now is 27 -th news
now is 28 -th news
now is 29 -th news
now is 30 -th news
now is 31 -th news
now is 32 -th news
now is 33 -th news
now is 34 -th news
now is 35 -th news
now is 36 -th news
now is 37 -th news
now is 38 -th news
now is 39 -th news
now is 40 -th news
now is 41 -th news
now is 42 -th news
now is 43 -th news
now is 44 -th news
now is 45 -th news
now is 46 -th news
now is 47 -th news
now is 48 -th news
now is 49 -th news
now is 50 -th news
now is 51 -th news
now is 52 -th news
now

,Char,Score
0,<s>,0.000000
1,▁,-0.007582
2,◎,-0.022131
3,学ぶ,-0.012538
4,育,-0.034943
...,...,...
230,しようと,-0.009714
231,していると,0.136439
232,話,0.036169
233,した,0.418090


## 评分后数据处理

In [ ]:
word_t = pd.DataFrame(columns=['text', 'text_new'])

In [ ]:
len_f = len(final_result1)
ij = 0
jn = 0
while(ij < len_f):
    if final_result1.iloc[ij, 0] == '<s>':
        txt = ''
        ij += 1
        ph_l = ''
        while(ij < len_f and final_result1.iloc[ij, 0] != '</s>'):
            txt += final_result1.iloc[ij, 0]
            ph_l += final_result1.iloc[ij, 0] + ' '
            ij += 1
        ph_l = ph_l.rstrip()
        word_t.loc[jn, :] = [txt, ph_l]
        jn += 1
    else:
        ij += 1
word_t 

,text,text_new
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...
1,▁◇単位円◆米ドル銀行間直物午後5時現在95.2395.26前営業日比△1.50高値94.9...,▁ ◇ 単位 円 ◆ 米 ドル 銀行 間 直 物 午後 5 時 現在 95 . 23 95 ...
2,▁ペプシカップ2002年世界少年野球選手権最終日12日・大阪ドームほか読売新聞大阪本社など主...,▁ ペ プ シ カップ 2002 年 世界 少年 野球 選手 権 最終 日 12 日 ・ 大...
3,▁2020年の東京五輪に向けポルトガル代表の陸上チームが大分市で事前キャンプをすることが決ま...,▁2020 年の 東京 五 輪 に 向け ポ ル ト ガ ル 代表 の 陸 上 チーム が ...
4,▁ワシントン中島健太郎米国の情報機関を統括するクラッパー国家情報長官は11日下院情報特別委員...,▁ ワ シン トン 中 島 健 太郎 米国 の情報 機関 を 統 括 する クラ ッ パー ...
...,...,...
1995,▁大蔵省は四日九三年の地域別国際収支を発表したそれによると対東南アジア中国など社会主義国家を...,▁大 蔵 省 は 四 日 九 三年 の 地域 別 国際 収 支 を 発表 した それ による...
1996,▁第11管区海上保安本部那覇市によると26日午後7時現在中国海警局の公船4隻が沖縄県石垣市の...,▁第 11 管 区 海上 保安 本 部 那 覇 市 によると 26 日 午後 7 時 現在 ...
1997,▁◇単位円◆米ドル銀行間直物午後5時現在109.36109.37前営業日比▼0.58高値10...,▁ ◇ 単位 円 ◆ 米 ドル 銀行 間 直 物 午後 5 時 現在 109 . 361 0...
1998,▁山口市の維新百年記念公園陸上競技場で10日に開催される第1回田島直人記念陸上競技大会読売新...,▁ 山 口 市の 維 新 百年 記念 公園 陸 上 競技 場 で 10 日に 開催される 第...


In [ ]:
final_df = final_result1[final_result1['Char'] != '<s>']
final_df = final_df[final_df['Char'] != '</s>']
final_df

,Char,Score
1,▁中国,-0.020270
2,遠,0.210349
3,征,-0.026880
4,へ,-0.013322
5,向,0.000466
...,...,...
289,く,0.011403
290,国際,0.001265
291,化,-0.010549
292,という,-0.001029


In [ ]:
word_t['Phrase']=word_t.text_new.apply(lambda x: x.split(' '))
df_t = word_t.explode("Phrase")

df_t

,text,text_new,Phrase
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,▁中国
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,遠
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,征
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,へ
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,向
...,...,...,...
1999,▁手元にわぴえ一〇年の歩みと題した冊子がある秋田県国際交流をすすめる婦人の会英語の略称をひら...,▁ 手元に わ ぴ え 一 〇 年の 歩 み と 題 した 冊 子 がある 秋 田 県 国際...,く
1999,▁手元にわぴえ一〇年の歩みと題した冊子がある秋田県国際交流をすすめる婦人の会英語の略称をひら...,▁ 手元に わ ぴ え 一 〇 年の 歩 み と 題 した 冊 子 がある 秋 田 県 国際...,国際
1999,▁手元にわぴえ一〇年の歩みと題した冊子がある秋田県国際交流をすすめる婦人の会英語の略称をひら...,▁ 手元に わ ぴ え 一 〇 年の 歩 み と 題 した 冊 子 がある 秋 田 県 国際...,化
1999,▁手元にわぴえ一〇年の歩みと題した冊子がある秋田県国際交流をすすめる婦人の会英語の略称をひら...,▁ 手元に わ ぴ え 一 〇 年の 歩 み と 題 した 冊 子 がある 秋 田 県 国際...,という


In [ ]:
df_t['Score'] = list(final_df.iloc[:, 1])
df_t

,text,text_new,Phrase,Score
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,▁中国,-0.020270
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,遠,0.210349
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,征,-0.026880
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,へ,-0.013322
0,▁中国遠征へ向けて合宿中のサッカー21歳以下U―21日本代表は3日反町新監督のもとで初めての...,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,向,0.000466
...,...,...,...,...
1999,▁手元にわぴえ一〇年の歩みと題した冊子がある秋田県国際交流をすすめる婦人の会英語の略称をひら...,▁ 手元に わ ぴ え 一 〇 年の 歩 み と 題 した 冊 子 がある 秋 田 県 国際...,く,0.011403
1999,▁手元にわぴえ一〇年の歩みと題した冊子がある秋田県国際交流をすすめる婦人の会英語の略称をひら...,▁ 手元に わ ぴ え 一 〇 年の 歩 み と 題 した 冊 子 がある 秋 田 県 国際...,国際,0.001265
1999,▁手元にわぴえ一〇年の歩みと題した冊子がある秋田県国際交流をすすめる婦人の会英語の略称をひら...,▁ 手元に わ ぴ え 一 〇 年の 歩 み と 題 した 冊 子 がある 秋 田 県 国際...,化,-0.010549
1999,▁手元にわぴえ一〇年の歩みと題した冊子がある秋田県国際交流をすすめる婦人の会英語の略称をひら...,▁ 手元に わ ぴ え 一 〇 年の 歩 み と 題 した 冊 子 がある 秋 田 県 国際...,という,-0.001029


In [ ]:
dff = df_t.groupby('text_new',sort=False)['Score'].apply(list).reset_index()
dff["index"]=range(len(dff))
dff

,text_new,Score,index
0,▁中国 遠 征 へ 向 けて 合 宿 中の サッカー 21 歳 以下 U ― 21 日本 代...,"[-0.020270460662178916, 0.21034879230814904, -...",0
1,▁ ◇ 単位 円 ◆ 米 ドル 銀行 間 直 物 午後 5 時 現在 95 . 23 95 ...,"[-0.0012030551294312198, -0.003579159027354587...",1
2,▁ ペ プ シ カップ 2002 年 世界 少年 野球 選手 権 最終 日 12 日 ・ 大...,"[-0.01359493575297121, -0.011264721364638547, ...",2
3,▁2020 年の 東京 五 輪 に 向け ポ ル ト ガ ル 代表 の 陸 上 チーム が ...,"[-0.00278674635272632, -0.015637769765457468, ...",3
4,▁ ワ シン トン 中 島 健 太郎 米国 の情報 機関 を 統 括 する クラ ッ パー ...,"[-0.015793309819371822, -0.005357804526023318,...",4
...,...,...,...
1995,▁大 蔵 省 は 四 日 九 三年 の 地域 別 国際 収 支 を 発表 した それ による...,"[-0.0470047989047801, -0.017756063618413233, -...",1995
1996,▁第 11 管 区 海上 保安 本 部 那 覇 市 によると 26 日 午後 7 時 現在 ...,"[-0.011516704146841876, -0.023121756653298003,...",1996
1997,▁ ◇ 単位 円 ◆ 米 ドル 銀行 間 直 物 午後 5 時 現在 109 . 361 0...,"[0.008390093303116946, 0.02691460321634827, 0....",1997
1998,▁ 山 口 市の 維 新 百年 記念 公園 陸 上 競技 場 で 10 日に 開催される 第...,"[0.00047933833981916494, -0.006638069871299532...",1998


In [ ]:
list_append=dff["Score"]
if len(df_sample)!=len(dff):
  dff.to_excel("/content/drive/MyDrive/情感打分数据包/日文/数据校对.xlsx")
  print("Please check")
df_sample["Score"]=list_append
df_sample

,txt,year,text_length,txt_seg,Topic,Doc_ID,Score_Negative,Score_Neutral,Score_Positive,nopunc,Score
0,中国遠征へ向けて合宿中のサッカー２１歳以下（Ｕ―２１）日本代表は３日、反町新監督のもとで初め...,2006.08.04,300,中国 遠征 向け 合宿 中 サッカー 歳 以下 日本 代表 日 反町 新 監督 もと 初めて...,24,48297,0.372833,0.429042,0.198125,中国遠征へ向けて合宿中のサッカー２１歳以下Ｕ―２１日本代表は３日反町新監督のもとで初めての練...,"[-0.020270460662178916, 0.21034879230814904, -..."
1,◇単位＝円◆米ドル（銀行間直物）午後５時現在９５．２３〜９５．２６前営業日比△１．５０高値９...,2008.11.21,317,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,18,48298,0.224488,0.666766,0.108745,◇単位円◆米ドル銀行間直物午後５時現在９５．２３９５．２６前営業日比△１．５０高値９４．９７...,"[-0.0012030551294312198, -0.003579159027354587..."
2,ペプシカップ２００２年世界少年野球選手権最終日（１２日・大阪ドームほか＝読売新聞大阪本社など...,2002.08.13,369,ペプシカップ 年 世界 少年 野球 選手権 最終 日 日 大阪ドーム ほか 読売新聞 大阪 ...,24,48301,0.204547,0.493237,0.302216,ペプシカップ２００２年世界少年野球選手権最終日１２日・大阪ドームほか読売新聞大阪本社など主催...,"[-0.01359493575297121, -0.011264721364638547, ..."
3,２０２０年の東京五輪に向け、ポルトガル代表の陸上チームが大分市で事前キャンプをすることが決ま...,2018.10.17,407,年 東京 五輪 向け ポルトガル 代表 陸上 チーム 大分 市 事前 キャンプ する こと ...,24,48303,0.271182,0.509268,0.219550,２０２０年の東京五輪に向けポルトガル代表の陸上チームが大分市で事前キャンプをすることが決まり...,"[-0.00278674635272632, -0.015637769765457468, ..."
4,【ワシントン＝中島健太郎】米国の情報機関を統括するクラッパー国家情報長官は１１日、下院情報特...,2013.04.12,553,ワシントン 中島 健太郎 米国 情報 機関 統括 する クラッパー 国家 情報 長官 日 下...,87,48308,0.438101,0.408954,0.152945,ワシントン中島健太郎米国の情報機関を統括するクラッパー国家情報長官は１１日下院情報特別委員会...,"[-0.015793309819371822, -0.005357804526023318,..."
...,...,...,...,...,...,...,...,...,...,...,...
1995,大蔵省は四日、九三年の地域別国際収支を発表した。それによると、対東南アジア（中国など社会主義...,1994.07.05,283,大蔵省 四 日 九 三 年 地域 別 国際 収支 発表 し た それ よる 対 東南アジア ...,99,56368,0.395083,0.452387,0.152530,大蔵省は四日九三年の地域別国際収支を発表したそれによると対東南アジア中国など社会主義国家を除...,"[-0.0470047989047801, -0.017756063618413233, -..."
1996,第１１管区海上保安本部（那覇市）によると、２６日午後７時現在、中国海警局の公船４隻が、沖縄県...,2017.09.27,85,第 管区 海上 保安 本部 那覇 市 よる 日 午後 時 現在 中国 海 警局 公 船 隻 ...,12,56381,0.286377,0.570069,0.143554,第１１管区海上保安本部那覇市によると２６日午後７時現在中国海警局の公船４隻が沖縄県石垣市の尖...,"[-0.011516704146841876, -0.023121756653298003,..."
1997,◇単位＝円◆米ドル（銀行間直物）午後５時現在１０９．３６〜１０９．３７前営業日比▼０．５８高...,2019.01.19,420,単位 円 米ドル 銀行 間 直 物 午後 時 現在 前 営業 日 比 高値 安値 前 営業 ...,8,56382,0.233884,0.639498,0.126618,◇単位円◆米ドル銀行間直物午後５時現在１０９．３６１０９．３７前営業日比▼０．５８高値１０９...,"[0.008390093303116946, 0.02691460321634827, 0...."
1998,山口市の維新百年記念公園陸上競技場で１０日に開催される第１回田島直人記念陸上競技大会（読売新...,2004.10.02,281,山口 市 維新 百 年 記念 公園 陸上 競技 場 日 開催 さ れる 第 回 田島 直人 ...,24,56388,0.355018,0.475439,0.169543,山口市の維新百年記念公園陸上競技場で１０日に開催される第１回田島直人記念陸上競技大会読売新聞...,"[0.00047933833981916494, -0.006638069871299532..."


## 保存数据

In [ ]:
df_sample.to_excel("/content/drive/MyDrive/情感打分数据包/日文/前14000日文分.xlsx",index=False)